# Compile results

In [1]:
import os
import zipfile

from tqdm.notebook import tqdm

In [2]:
import itertools
import json
import warnings

from numpy import arange
from numpy.random import default_rng

In [3]:
from pNeuma_simulator import params
from pNeuma_simulator.results import aggregate, percolate, loader

In [4]:
warnings.filterwarnings("ignore")

In [5]:
path = "./output/"
os.makedirs(path, exist_ok=True)
# Combinatorial configurations
n_veh = 10
scale = 2
l_cars = scale * arange(1, n_veh, 1)
l_moto = scale * arange(0, n_veh - 1, 1)
permutations = list(itertools.product(l_cars, l_moto))
len(permutations)

81

In [6]:
# Pseudorandom number generator
seed = 1024
rng = default_rng(seed)
compile = True

In [7]:
path = "./output/"
os.makedirs(path, exist_ok=True)
with open(f"{path}results_r2.json") as openfile:
    results = json.load(openfile)

In [8]:
n_cars = 10
n_moto = 12
permutation = (n_cars, n_moto)

In [9]:
# def loader(permutation, verbose=True):
verbose = True
items = []
accidents = 0
with zipfile.ZipFile(f"{path}{permutation}_r.zip", "r") as ziph:
    # ziph is zipfile handle
    for filename in ziph.namelist():
        if filename.endswith(").jsonl"):
            # Opening JSONL file
            with ziph.open(filename, "r") as openfile:
                # Reading from JSONL file
                # https://stackoverflow.com/questions/2081836/
                for i, line in enumerate(tqdm(openfile)):
                    item = json.loads(line)
                    if isinstance(item[0], list):
                        items.append(item)
                    else:
                        accidents += 1
                if verbose:
                    print(openfile.name)
#   return items

0it [00:00, ?it/s]

(10, 12).jsonl


In [10]:
accidents

0

In [11]:
if compile:
    # results = {}
    for permutation in [(n_cars, n_moto)]:  # permutations:
        # n_cars, n_moto = permutation
        results[str(permutation)] = {
            "VKT_cars": [],
            "VHT_cars": [],
            "VKT_moto": [],
            "VHT_moto": [],
        }
        # items = loader(permutation, path)
        for item in items:
            (
                VKT_cars,
                VHT_cars,
                VKT_moto,
                VHT_moto,
            ) = aggregate(item[0], n_cars, n_moto)
            results[str(permutation)]["VKT_cars"].append(VKT_cars)
            results[str(permutation)]["VHT_cars"].append(VHT_cars)
            results[str(permutation)]["VKT_moto"].append(VKT_moto)
            results[str(permutation)]["VHT_moto"].append(VHT_moto)
        error = accidents / (accidents + len(items))
        results[str(permutation)]["error"] = error
        if n_cars > 2 and n_moto > 2:
            percolation = percolate(items, n_cars, n_moto, rng, start=50)
        else:
            percolation = None
        results[str(permutation)]["percolation"] = percolation
        items = None

In [12]:
write = compile
if write:
    with open(f"{path}results_r2.json", "w") as outfile:
        json.dump(results, outfile)